In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import notebook_helper
import sys
sys.path.append('../experiments/')
results_dir = '../results/'

Load the results into a pandas dataframe

In [2]:
dir_names = sorted([fname
                    for fname in os.listdir(results_dir)
                    if os.path.isdir(join(results_dir, fname))
                    and os.path.exists(join(results_dir, fname, 'results.pkl'))
                    ])
results_list = []
for dir_name in tqdm(dir_names):
    ser = pd.Series(
        joblib.load(join(results_dir, dir_name, 'results.pkl')))
    results_list.append(ser)

r = pd.concat(results_list, axis=1).T.infer_objects()
r.to_pickle(join(results_dir, 'results_aggregated.pkl'))

100%|██████████| 3/3 [00:00<00:00, 1721.09it/s]


In [3]:
r

,dataset_name,seed,save_dir,checkpoint,n_shots,use_cache,save_dir_unique,resps,precision,recall
0,medication_status,1,/home/chansingh/clinical-extract-and-check/res...,gpt-4-0314,0,1,/home/chansingh/clinical-extract-and-check/res...,[Gatifloxacin: initiated for encapsulated orga...,0.075340,0.092540
1,medication_status,1,/home/chansingh/clinical-extract-and-check/res...,gpt-4-0314,1,1,/home/chansingh/clinical-extract-and-check/res...,"[""Gatifloxacin"" (initiated)\n- ""Acyclovir"" (pr...",0.781882,0.873197
2,medication_status,1,/home/chansingh/clinical-extract-and-check/res...,gpt-4-0314,5,1,/home/chansingh/clinical-extract-and-check/res...,"[""gatifloxacin"" (active)\n- ""acyclovir"" (activ...",0.872925,0.890113


In [4]:
r.iloc[0]['resps']

['Gatifloxacin: initiated for encapsulated organism prophylaxis at discharge\n- Acyclovir: prophylactic therapy through day 100\n- Bactrim: active for PCP prophylaxis\n- Systemic steroids: weaned (discontinued)\n- Cyclosporin: discontinued during last admission',
 'Levothyroxine 0.15 mg p.o. q.d.: Discontinued',
 'Azithromycin: Discontinued (after a full 7-day course)\n- Clindamycin: Active\n- Ceftriaxone: Discontinued (after a full 7-day course)',
 'Lipitor: discontinued\n- Zocor: active\n- Haldol: active\n- TSH: neither (test result)\n- T3: neither (test result)\n- Free T4: neither (test result)',
 'NS 75 cc/h for 6 hours only: Active\n- Hold lasix today: Discontinued\n- Discontinue lisinopril: Discontinued\n- Continue metoprolol: Active\n- EC ASA 81 mg QD: Active',
 'Darvocet: Active\n- Cozaar: Resuming (Active)\n- IV fluids: Discontinued',
 'Zithromax: Active (will add atypical coverage)\n- Levaquin: Discontinued',
 'Fentanyl patch 75 micrograms (active)\n- Percocet (discontinued)\

In [ ]:
# fill missing args with default values from argparse
r = notebook_helper.fill_missing_args_with_default(r, fname='01_train_model.py')

# group using these experiment hyperparams when averaging over random seeds
group_keys = [k for k in notebook_helper.get_main_args_list(fname='01_train_model.py') if not k == 'seed']
ravg = (
    r
    .groupby(by=group_keys)
    .mean()
    .reset_index()
)

# apply cross validation
ravg_cv = (
    ravg
    .sort_values(by='accuracy_cv', ascending=False)
    .groupby(by=['model_name', 'dataset_name'])
    .first()
    .reset_index()
)
ravg_cv

In [ ]:
sns.barplot(data=ravg_cv, x='model_name', y='accuracy_test') #, hue='dataset_name')
plt.show()

Load an individual model

In [ ]:
run_args = r.iloc[0]
model = joblib.load(join(run_args.save_dir_unique, 'model.pkl'))
model